In [12]:
import numpy as np
import random
from glob import glob

#We're using function from image_recognition to process photos
# from image_recognition import

#This is where the classification and categorization of images will occur
data_dir = 'Dataset'
train = 'Dataset/training'
test = 'Dataset/evaluation'
valid = 'Dataset/validation'
label_names = ['Bread', 'Dairy Product', 'Dessert', 'Egg', 'Fried Food', 'Meat', 'Noodles-Pasta', 'Rice',
               'Seafood', 'Soup', 'Vegetable-Fruit']

training_files = glob('Dataset/training/*/*.jpg')
training_set_size = [50,100, 250, 500, 1000, 5000, 9000]

new_training_set = []

#This will split the training set into different sizes
def split_training_size(files, n):
    random_set = random.sample(training_files, k=n)
    for i in range(len(random_set)):
        new_training_set.append(random_set[i])
    return new_training_set


split_training_size(training_files, training_set_size[0])
for i in range(len(new_training_set)):
    print("This is element " + str(i) + ": " + str(new_training_set[i]))

50
This is element 0: Dataset/training/Meat/486.jpg
This is element 1: Dataset/training/Dairy product/155.jpg
This is element 2: Dataset/training/Dessert/521.jpg
This is element 3: Dataset/training/Egg/330.jpg
This is element 4: Dataset/training/Fried food/359.jpg
This is element 5: Dataset/training/Fried food/663.jpg
This is element 6: Dataset/training/Bread/349.jpg
This is element 7: Dataset/training/Vegetable-Fruit/460.jpg
This is element 8: Dataset/training/Bread/298.jpg
This is element 9: Dataset/training/Vegetable-Fruit/15.jpg
This is element 10: Dataset/training/Fried food/825.jpg
This is element 11: Dataset/training/Soup/295.jpg
This is element 12: Dataset/training/Egg/731.jpg
This is element 13: Dataset/training/Dessert/273.jpg
This is element 14: Dataset/training/Dessert/998.jpg
This is element 15: Dataset/training/Meat/1294.jpg
This is element 16: Dataset/training/Dairy product/179.jpg
This is element 17: Dataset/training/Noodles-Pasta/113.jpg
This is element 18: Dataset/tra

In [ ]:
import os
# tensorflow
import tensorflow.keras as keras
import tensorflow as tf

# image processing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

# model / neural network
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

datagen = ImageDataGenerator()
train_generator = datagen.flow_from_directory( 
    directory=train, 
    classes = label_names,
    target_size=(512, 512),  
    batch_size=50, 
    class_mode="binary", 
)
# validation data
valid_generator = datagen.flow_from_directory( 
    directory=valid, 
    classes = label_names,
    target_size=(512, 512), 
    batch_size=50, 
    class_mode="binary", 
    shuffle = True
)

    # build the entire model
    # Code taken from 
    # https://github.com/ovh/ai-training-examples/blob/main/notebooks/computer-vision/image-classification/tensorflow/resnet50/notebook-resnet-transfer-learning-image-classification.ipynb
    # Created by eleapttn
def model_creation():
    resnet_50 = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
    for layer in resnet_50.layers:
        layer.trainable = False
    x = resnet_50.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x) 
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x) 
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x) 
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='relu')(x) 
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(11, activation='softmax')(x)
    return (Model(inputs = resnet_50.input, outputs = predictions))

# define training function
    # Code taken from 
    # https://github.com/ovh/ai-training-examples/blob/main/notebooks/computer-vision/image-classification/tensorflow/resnet50/notebook-resnet-transfer-learning-image-classification.ipynb
    # Created by eleapttn
def trainModel(model, passes, optimizer):
    batch_size = 32
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model.fit(train_generator, validation_data=valid_generator, epochs=passes, batch_size=batch_size)

In [26]:
model = model_creation()
model_history = trainModel(model, 10, "Adam")

AttributeError: 'NoneType' object has no attribute 'compile'

In [22]:
test_data = datagen.flow_from_directory( 
    directory=test, 
    classes = label_names,
    target_size=(224, 224), 
    batch_size=32, 
    class_mode="binary",
    shuffle=False
)


def find_accuracy(labels, predictions):
    sum = 0
    for i in range(len(labels)):
        if labels[i]==predictions[i]:
            sum +=1
    return sum/len(labels)


test_class = test_data.labels
print(test_class)
training_Accuracy = []
validation_Accuracy = []
testing_Accuracy = []
for set in training_sets:
    model = model_creation()
    model_history = trainModel(model, passes=10, optimizer="Adam")
    predictions = model.predict(test_data)
    class_ids = np.argmax(predictions, axis=1)
    training_Accuracy.append(find_accuracy(test_class, class_ids))
    validation_Accuracy.append(model_history)
    

Found 3347 images belonging to 11 classes.
[ 0  0  0 ... 10 10 10]


AttributeError: 'NoneType' object has no attribute 'predict'